# IBM Applied Data Science Capstone Project
## Exploring Dog Friendly Neighborhoods in Manhattan

## Table of contents
* Introduction
* Data
* Methodology
* Analysis
* Results and Discussion
* Conclusion



## Introduction

The city can be a tricky place to raise a dog. Dogs need open spaces, accepting communities, pet supply and grooming locations, and plenty of dog-friendly establishments and eateries. That’s why dog owners are always looking for the perfect neighborhood to raise their pup, and it can be challenging to find. Manhattan is one of the more dog-welcoming cities, but what is the perfect area in Manhattan for a dog owner? This analysis will find a solution to the problem by identifying the top dog-friendly neighborhoods to live in Manhattan.

## Data <a name="data"></a>

A dataset of New York City neighborhoods, as well as their latitude and longitude, will be used to create a database of neighborhoods in Manhattan to analyze (https://geo.nyu.edu/catalog/nyu_2451_34572). In addition, Foursquare location data will be used to find the neighborhoods that provide the most extensive list of top dog related establishments (https://developer.foursquare.com). Venues identified as dog-friendly include Dog Run, Outdoors & Recreation, Park, Pet Café, Pet Service, Pet Store, Trail, and Veterinarian. Finally, neighborhoods will be rated using a ‘Dog Friendly Factor’ based on the frequency of the dog-friendly venues identified above within the neighborhood. 

First let's download everything we need.

## Methodology <a name="data"></a>

First the data was downloaded and cleaned. After downloading the New York City dataset, information regarding Borough, Neighborhood, Latitude, and Longitude were pulled and transformed into a pandas dataframe. The resulting dataframe included 5 boroughs and 306 neighborhoods. A geopy library was used to get the latitude and longitude values of New York City and create a map of the neighborhoods from the dataset. 
Next, in order to focus on Manhattan neighborhoods, the New York City dataframe was sliced to create a new dataframe of Manhattan data only. A similar process was used here, identifying the latitude and longitude values of Manhattan and creating a map of Manhattan only neighborhoods.
Once the Manhattan neighborhoods to be analyzed had been visualized, Foursquare API was used to evaluate and segment the neighborhoods. Using a radius of 1000 meters, the top 50 venues in each neighborhood were identified and structured into a pandas dataframe. There were 277 unique venues identified. The final step in creating the aggregate database was grouping each row by neighborhood and taking the mean of the frequency of occurrence of each venue. 
Lastly, dog-friendly venue frequencies were identified and combined in a new database. As mentioned above, these venues were assumed to include Dog Run, Outdoors & Recreation, Park, Pet Café, Pet Service, Pet Store, Trail, and Veterinarian. The ‘Dog Friendly Factor’ was defined as the sum of frequencies across all dog-friendly venues for each neighborhood. Adding this factor to the database created a means to rank the neighborhoods. Adjusting the final database by sorting on the ‘Dog Friendly Factor’ in descending order identified the top dog-friendly neighborhoods.

## Analysis <a name="data"></a>

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset

Next, download the New York City dataset to access longitude and latitude information for the various neighborhoods.

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [5]:
neighborhoods_data = newyork_data['features']

In [6]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] # define the dataframe columns
neighborhoods = pd.DataFrame(columns=column_names) # instantiate the dataframe

Now let's create the pandas dataframe of New York City neighborhood data.

In [7]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [8]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [9]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


Use geopy to find the latitude and longitude of NYC.

In [10]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

Now we can create a map of NYC.

In [11]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


Now we can do the same thing for Manhattan only.

In [12]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


In [13]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [14]:
CLIENT_ID = 'EKVRVXSOVRUUFWGADFRCZXIVODSLLSYJQ5I0Y0TGRLWKSJ4H' # Foursquare ID
CLIENT_SECRET = 'R2A1CTMQGZXNYHDXAM30J2HEVLZRN4F35AFYDPGZCZWU3UP2' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

Next we need to define our Foursquare credentials in order to use the Foursquare API.

The 10 neighborhoods listed above are the top dog-friendly neighborhoods in Manhattan according to this analysis!

Given we know that all the information is in the 'items' key, let's use that to create a function that extracts the category of the venue.

In [22]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Next let's use the GET request and structure the Foursquare neighborhood data into a pandas dataframe.

In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )



Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


Explore the dataset.

In [35]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3317, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


In [36]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,100,100,100,100,100,100
Carnegie Hill,100,100,100,100,100,100
Central Harlem,43,43,43,43,43,43
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,41,41,41,41,41,41
East Village,100,100,100,100,100,100


In [37]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 332 uniques categories.


In [38]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Gym,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Ski Shop,

The next step is to find the frequency of each venue per neighborhood .

In [39]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Gym,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Ski Shop,

Now we isolate the frequencies of dog-friendly venues only and create a ranking factor to sort by overall frequency of dog-friendly venues. I have decided to use the frequency sum across all dog-friendly categories as a relative metric.

In [50]:
#frequency of dog friendly venues
dogfriendly=manhattan_grouped[['Neighborhood','Dog Run','Outdoors & Recreation','Park','Pet Café','Pet Service','Pet Store','Trail','Veterinarian']]
dogfriendly.head()

,Neighborhood,Dog Run,Outdoors & Recreation,Park,Pet Café,Pet Service,Pet Store,Trail,Veterinarian
0,Battery Park City,0.0,0.0,0.080000,0.0,0.0,0.01,0.0,0.0
1,Carnegie Hill,0.0,0.0,0.000000,0.0,0.0,0.01,0.0,0.0
2,Central Harlem,0.0,0.0,0.023256,0.0,0.0,0.00,0.0,0.0
3,Chelsea,0.0,0.0,0.010000,0.0,0.0,0.01,0.0,0.0
4,Chinatown,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0


In [51]:
dogfriendly_ranked = dogfriendly.copy()
dogfriendly_ranked['Dog Friendly Factor'] = dogfriendly_ranked[dogfriendly_ranked.columns].sum(axis=1)
dogfriendly_ranked.head()

,Neighborhood,Dog Run,Outdoors & Recreation,Park,Pet Café,Pet Service,Pet Store,Trail,Veterinarian,Dog Friendly Factor
0,Battery Park City,0.0,0.0,0.080000,0.0,0.0,0.01,0.0,0.0,0.090000
1,Carnegie Hill,0.0,0.0,0.000000,0.0,0.0,0.01,0.0,0.0,0.010000
2,Central Harlem,0.0,0.0,0.023256,0.0,0.0,0.00,0.0,0.0,0.023256
3,Chelsea,0.0,0.0,0.010000,0.0,0.0,0.01,0.0,0.0,0.020000
4,Chinatown,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000


In [53]:
dogfriendly_neighborhood = dogfriendly_ranked.sort_values('Dog Friendly Factor', ascending=False)
dogfriendly_neighborhood.head(10)

,Neighborhood,Dog Run,Outdoors & Recreation,Park,Pet Café,Pet Service,Pet Store,Trail,Veterinarian,Dog Friendly Factor
30,Stuyvesant Town,0.000000,0.000000,0.105263,0.0,0.052632,0.000000,0.000000,0.000000,0.157895
33,Tudor City,0.036585,0.000000,0.060976,0.0,0.000000,0.012195,0.012195,0.000000,0.121951
28,Roosevelt Island,0.038462,0.038462,0.038462,0.0,0.000000,0.000000,0.000000,0.000000,0.115385
15,Inwood,0.017544,0.000000,0.035088,0.0,0.000000,0.017544,0.017544,0.017544,0.105263
0,Battery Park City,0.000000,0.000000,0.080000,0.0,0.000000,0.010000,0.000000,0.000000,0.090000
25,Morningside Heights,0.000000,0.000000,0.071429,0.0,0.000000,0.000000,0.000000,0.000000,0.071429
32,Tribeca,0.010000,0.000000,0.050000,0.0,0.000000,0.000000,0.000000,0.000000,0.060000
14,Hudson Yards,0.027397,0.000000,0.027397,0.0,0.000000,0.000000,0.000000,0.000000,0.054795
34,Turtle Bay,0.010000,0.000000,0.030000,0.0,0.000000,0.010000,0.000000,0.000000,0.050000
17,Lincoln Square,0.010000,0.000000,0.030000,0.0,0.000000,0.010000,0.000000,0.000000,0.050000


The 10 neighborhoods listed above are the top dog-friendly neighborhoods in Manhattan!

## Results <a name="results"></a>

## Discussion <a name="results"></a>

The analysis enabled a breakdown of which neighborhoods are best suited for a dog owner. Venues such as parks, pet cafés, veterinarians, and other dog-related locations were analyzed to create a metric for dog-friendliness. There are several items that should be taken into account while digesting these results: 1) Assumptions were made, for example assuming that all ‘Parks’ identified here are dog-friendly; 2) This was not an exhaustive list of dog-friendly venues available to dog owners, and only those identified in the dataset available were used; and 3) The ‘Dog Friendly Factor’ was a simple metric and a more complex metric could be used to achieve the next level of accuracy.

## Conclusion <a name="conclusion"></a>

This analysis involved an initial review of dog-friendly neighborhoods in Manhattan and provided several great options, including Stuyvesant Town, Tudor City, Roosevelt Island, and Inwood among the top picks. Future analysis could build upon this base by including an even more detailed list of dog-friendly venues and constructing a more complex statistical metric for ranking. Furthermore, the analysis could be adapted to evaluate additional cities around the world.